# Explore AuxTel Prompt Processing Subtractions
Michael Wood-Vasey  
Started: 2023-08-16

Goals:
1. Locate and load sample AuxTel run from recent observations in 2023 July
2. Visualize an image
3. Find a subtraction.
4. Display template, science, subtraction
5. Check out performance of DIA
6. Write note with 1-3 suggestions for improvement of DIA

References
1. Track Prompt Processing AuxTel runs here  
    https://confluence.lsstcorp.org/display/DM/Prompt+Processing+with+AuxTel+Imaging+Survey+Data+2023  
    
LATISS/prompt/output-2023-05-24/ApPipe/prompt-proto-service-00026  
LATISS/prompt/output-2023-08-15/ApPipe/prompt-proto-service-00010  


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re

In [ ]:
import lsst.afw.display as afwDisplay
from lsst.analysis.tools.actions.vector import ConvertUnits, FlagSelector, GoodDiaSourceSelector, LoadVector
from lsst.analysis.tools.atools import SkySourceHistPlot, SkySourceSkyPlot, NumDiaSourcesAllMetric, NumDiaSourcesMetric, NumDipolesMetric
from lsst.daf.butler import Butler
import lsst.geom as geom

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
%matplotlib inline

In [ ]:
repo = "/sdf/group/rubin/repo/embargo/"

In [ ]:
butler = Butler(repo)

I here use the command-line `butler query-collections` to get the info.  I couldn't figure out the in-Python Butler equivalent.

Here are the DRP-type processing for 2023 August

  I pass "LATISS\*runs\*2023-08\*" to get only collections from LATISS that have dates from 2023 August.

In [ ]:
! butler query-collections /sdf/group/rubin/repo/embargo "LATISS*runs*2023-08*"

In [ ]:
# Neven - This produces similar, but not identical result via python
# code here: https://github.com/lsst/daf_butler/blob/11710eee44873e6cf90023bbc40d1745b61480a4/python/lsst/daf/butler/_registry_shim.py#L280
butler.registry.queryCollections("LATISS*runs*2023-08*", flattenChains=True, includeChains=True)

And here's the AP-ish Prompt Processing (prototype) collection

In [ ]:
! butler query-collections /sdf/group/rubin/repo/embargo "LATISS/prompt/*-2023-08*/ApPipe/*"

In [ ]:
ap_collection = "LATISS/prompt/output-2023-08-15/ApPipe/prompt-proto-service-00010"
drp_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-08A/w_2023_19/PREOPS-3613"

In [ ]:
! butler query-dataset-types /sdf/group/rubin/repo/embargo

In [ ]:
! butler query-datasets /sdf/group/rubin/repo/embargo \
    --collections "LATISS/runs/AUXTEL_DRP_IMAGING_2023-08A/w_2023_19/PREOPS-3613" \
    --where "instrument='LATISS' and detector=0 and band='r' and visit > 2023080100000 and visit < 2023080500000"  \
    calexp

In [ ]:
! butler query-datasets /sdf/group/rubin/repo/embargo \
    --collections "LATISS/runs/AUXTEL_DRP_IMAGING_2023-08A/w_2023_19/PREOPS-3613" \
    --where "instrument='LATISS' and detector=0 and band='r' and visit > 2023080100000 and visit < 2023080500000"  \
    deepDiff_differenceExp

In [ ]:
! butler query-datasets /sdf/group/rubin/repo/embargo \
    --where "instrument='LATISS' and detector=0 and band='r' and visit > 2023080100000 and visit < 2023080500000"  \
    deepDiff_differenceExp

In [ ]:
! butler query-datasets /sdf/group/rubin/repo/embargo \
    --collections "LATISS/prompt/output-2023-08-15/ApPipe/prompt-proto-service-00010" \
    --where "instrument='LATISS' and detector=0 and band='r' and visit > 2023080100000 and visit < 2023082500000" \
    deepDiff_differenceExp

In [ ]:
! butler query-datasets /sdf/group/rubin/repo/embargo \
    --collections "LATISS/prompt/output-2023-08-15/ApPipe/prompt-proto-service-00010" \
    --where "instrument='LATISS' and detector=0 and band='r' and visit > 2023080100000 and visit < 2023082500000" 

In [ ]:
# DRP 2023-08-03 reduction
# data_id = {"instrument": "LATISS", "detector": 0, "band": "r", "visit": 2023080300670}
# drp_collection

# AP 2023-08-15 reduction
# ap_collection
data_id = {"instrument": "LATISS", "detector": 0, "band": "r", "visit": 2023081500112}

In [ ]:
calexp = butler.get("calexp", dataId=data_id, collections=ap_collection)

In [ ]:
calexp

In [ ]:
fig = plt.figure(figsize=(12, 10))
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(calexp.image)
plt.show()

In [ ]:
diff = butler.get("deepDiff_differenceExp", dataId=data_id, collections=ap_collection)

In [ ]:
# Includes detections and sky sources. Direct output from detect+measure after image subtraction.
diaSrc = butler.get("deepDiff_diaSrc", dataId=data_id, collections=ap_collection)
# The SDM/DPDD (Science Data Model/Data Products Definition Document) version of the diaSrc Table. No sky sources.  Flags are packed into uint64.
diaSrcTable = butler.get("deepDiff_diaSrcTable", dataId=data_id, collections=ap_collection)

In [ ]:
diaSrcTable["flags"]

In [ ]:
PIXEL_FLAG_LIST = [
    'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge', 'base_PixelFlags_flag_interpolated',
    'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_bad',
    'base_PixelFlags_flag_suspect', 'base_PixelFlags_flag_interpolatedCenter', 'base_PixelFlags_flag_saturatedCenter',
    'base_PixelFlags_flag_crCenter', 'base_PixelFlags_flag_suspectCenter'
]
SHAPE_FLAG_LIST = ["slot_Shape_flag"]
EDGE_FLAG_LIST = ["base_NaiveCentroid_flag_edge", "base_SdssCentroid_flag_edge"]
SKY_SOURCE = ["sky_source"]

FLAG_LIST = PIXEL_FLAG_LIST + SHAPE_FLAG_LIST + EDGE_FLAG_LIST + SKY_SOURCE

SNR_THRESHOLD = 5.5

def get_good_sources_idx(df, flag_list=FLAG_LIST, snr_threshold=SNR_THRESHOLD):
    bad = np.array(np.zeros_like(df), dtype=bool)
    for flag in flag_list:
        bad |= df[flag]

    snr = df["slot_ApFlux_instFlux"] / df["slot_ApFlux_instFluxErr"]
    # This is a diff so take things both above the positive SNR cut and below the negative SNR cut
    good_snr = (snr < -snr_threshold) | (snr > snr_threshold)
        
    good = ~bad
    good &= good_snr
    return good

In [ ]:
good_source_idx = get_good_sources_idx(diaSrc)

In [ ]:
good_sources = diaSrc.asAstropy()[good_source_idx]

In [ ]:
len(good_sources)

In [ ]:
template = butler.get("deepDiff_templateExp", dataId=data_id, collections=ap_collection)
matched_template = butler.get("deepDiff_matchedExp", dataId=data_id, collections=ap_collection)

In [ ]:
fig = plt.figure(figsize=(12, 12))
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(template.image)
for x, y in good_sources[["slot_Centroid_x", "slot_Centroid_y"]]:
    display.dot("o", x, y, ctype="green", size=20)
plt.title('Template')
plt.show()


In [ ]:
fig = plt.figure(figsize=(12, 12))
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(diff.image)
for x, y in diaSrc.asAstropy()[["slot_Centroid_x", "slot_Centroid_y"]]:
    display.dot("o", x, y, ctype="red", size=20)
for x, y in good_sources[["slot_Centroid_x", "slot_Centroid_y"]]:
    display.dot("o", x, y, ctype="green", size=20)
plt.title('Difference image')
plt.show()

In [ ]:
[c for c in good_sources.columns if re.search("slot.*flag.*", c)]

In [ ]:
flags = PIXEL_FLAG_LIST + ["base_PixelFlags_flag"]
flags

## Make postage templ, sci, diff postage stamps for each of the good_sources

In [ ]:
def show_cutout(exposure, x, y, size=30):
    """
    Create and show a cutout of an exposure given a position
    """
    xy = geom.PointI(x, y)
    cutoutSize = geom.ExtentI(size, size)
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    cutout = exposure.getCutout(bbox)

    fig = plt.figure()
    plt.imshow(cutout.image.array, cmap="gray", origin='lower')

In [ ]:
def show_cutout_triplet(template, science, diff, x, y, size=30, figsize=(8, 3)):
    """
    Create and show a cutout of an template, science and difference image
    given a position
    """
    xy = geom.PointI(x, y)
    cutoutSize = geom.ExtentI(size, size)
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

    template_cutout = template.getCutout(bbox)
    science_cutout = science.getCutout(bbox)
    diff_cutout = diff.getCutout(bbox)

    # Use diff sigma for calexp, diff ranges.
    sigma = np.sqrt(np.median(diff.variance.array))

    fit, axes = plt.subplots(1, 3, figsize=figsize)
    plt.sca(axes[0])
    plt.imshow(template_cutout.image.array, cmap="gray", origin='lower')
    plt.title("Template")
    plt.sca(axes[1])
    plt.imshow(science_cutout.image.array, cmap="gray", vmin=-5*sigma, vmax=+5*sigma, origin='lower')
    plt.title("Science")
    plt.sca(axes[2])
    plt.imshow(diff_cutout.image.array, cmap="gray", vmin=-5*sigma, vmax=+5*sigma, origin='lower')
    plt.title("Diff")

In [ ]:
stamp_size = 30 # pixels.  Size of baseline alert stamp
for x, y in good_sources[["slot_Centroid_x", "slot_Centroid_y"]]:
    print(x, y)
    show_cutout_triplet(matched_template, calexp, diff, x, y, size=stamp_size)

## Check the performance of the sky sources

In [ ]:
# What is the difference between sky_sources and diaSrc?
# Sky sources are implemented in exactly the same manner as sky objects, with up to nSources sky sources randomly placed for any given CCD.
# Sky objects are pseudo-objects injected into the data processing pipeline 
# https://community.lsst.org/t/sky-sources-added-to-single-frame-processing/4137
sky_sources = diaSrc.asAstropy()
sky_sources = sky_sources[sky_sources["sky_source"]]  

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))  

plt.sca(axes[0])
bins = np.linspace(-100, +100)
hkwargs = {"bins": bins, "histtype": "step"}
# plt.hist(diaSrc["slot_PsfFlux_instFlux"] / diaSrc["slot_PsfFlux_instFluxErr"], label="InstFlux SNR (all)", **hkwargs)
# plt.hist(good_sources["slot_PsfFlux_instFlux"] / good_sources["slot_PsfFlux_instFluxErr"], label="InstFlux SNR (good)", **hkwargs)
plt.hist(diaSrc["slot_ApFlux_instFlux"] / diaSrc["slot_ApFlux_instFluxErr"], label="All", **hkwargs)
plt.hist(good_sources["slot_ApFlux_instFlux"] / good_sources["slot_ApFlux_instFluxErr"], label="Good", **hkwargs)
plt.hist(sky_sources["slot_ApFlux_instFlux"] / sky_sources["slot_ApFlux_instFluxErr"], label="Sky", **hkwargs)
plt.yscale("log")
plt.title("ApFlux SNR")
plt.legend(loc="upper left");

# Detection should be at 3-sigma
# Let's zoom in and see
plt.sca(axes[1])
bins = np.linspace(-10, +10)
hkwargs = {"bins": bins, "histtype": "step"}
# plt.hist(diaSrc["slot_PsfFlux_instFlux"] / diaSrc["slot_PsfFlux_instFluxErr"], label="InstFlux SNR (all)", **hkwargs)
# plt.hist(good_sources["slot_PsfFlux_instFlux"] / good_sources["slot_PsfFlux_instFluxErr"], label="InstFlux SNR (good)", **hkwargs)
plt.hist(diaSrc["slot_ApFlux_instFlux"] / diaSrc["slot_ApFlux_instFluxErr"], label="All", **hkwargs)
plt.hist(good_sources["slot_ApFlux_instFlux"] / good_sources["slot_ApFlux_instFluxErr"], label="Good", **hkwargs)
plt.hist(sky_sources["slot_ApFlux_instFlux"] / sky_sources["slot_ApFlux_instFluxErr"], label="Sky", **hkwargs)
print(np.std(sky_sources["slot_ApFlux_instFlux"] / sky_sources["slot_ApFlux_instFluxErr"]))

from lsst.analysis.tools.statistics import nansigmaMad
print(nansigmaMad(sky_sources["slot_ApFlux_instFlux"] / sky_sources["slot_ApFlux_instFluxErr"]))

plt.axvline(3, color="grey", linestyle="--", label="Detection threshold")
plt.axvline(-3, color="grey", linestyle="--")
# where does 5 sigma come from?
plt.axvline(5, color="grey", linestyle="-.", label="Good threshold")
plt.axvline(-5, color="grey", linestyle="-.")
plt.title("ApFlux SNR zoom")
plt.legend();


## Analysis Tools

Generate the analysis tools plots and statistics

Do this directly in the Notebook, following analysis_tools_examples

In [ ]:
plot_info_dict = {"bands": data_id["band"], "visit": data_id["visit"], "run": ap_collection, "tableName": "deepDiff_diaSrc"}

In [ ]:
plot = SkySourceHistPlot()

In [ ]:
# Use the GoodDiaSourceSelector to get the standard pixel flag selection of good DIA sources
plot.prep.selectors.skySourceSelector = GoodDiaSourceSelector
# And then specify that we just want the sky sources
plot.prep.selectors.skySourceSelector.selectWhenTrue = ["sky_source"]

# We have to rename the columns from the SkySource assumptions of having a DPDD-ified catalog
plot.process.buildActions.hist_psf_flux.vectorKey = "slot_PsfFlux_instFlux"
plot.process.buildActions.hist_ap09_flux.vectorKey = "base_CircularApertureFlux_9_0_instFlux"
plot.process.buildActions.hist_psf_sn.fluxType = "slot_PsfFlux_instFlux"
plot.process.buildActions.hist_ap09_sn.fluxType = "base_CircularApertureFlux_9_0_instFlux"

In [ ]:
plot.produce.plot.panels["panel_flux"].bins = 25
plot.produce.plot.panels["panel_sn"].bins = 25

In [ ]:
plot.finalize()
results = plot(diaSrc, band=data_id["band"], plotInfo=plot_info_dict)

This pipetasks wouldn't work because apCcdVisitQualityCore.yaml requires fakes_deepDiff_assocDiaSrc and we don't have the assocDiaSrc files.

! pipetask run -p apCcdVisitQualityCore.yaml \
-b /sdf/group/rubin/repo/embargo \
-i "LATISS/prompt/output-2023-08-15/ApPipe/prompt-proto-service-00010" \
-o u/wmwv/newDiaPlot \
--register-dataset-types

In [ ]:
plot = SkySourceSkyPlot()

# Use the GoodDiaSourceSelector to get the standard pixel flag selection of good DIA sources
plot.prep.selectors.skySourceSelector = GoodDiaSourceSelector
# And then specify that we just want the sky sources
plot.prep.selectors.skySourceSelector.selectWhenTrue = ["sky_source"]

# We have to rename the columns from the SkySource assumptions of having a DPDD-ified catalog
plot.process.buildActions.z.vectorKey = "base_CircularApertureFlux_9_0_instFlux"
plot.process.buildActions.statMask.fluxType = "slot_PsfFlux_instFlux"

# Convert from rad to degree
# I think this is the simplest way to do it
# We have to override the existing x, y VectorActions
plot.process.buildActions.x = ConvertUnits(buildAction=LoadVector, inUnit="rad", outUnit="degree")
plot.process.buildActions.x.buildAction.vectorKey = "coord_ra"
plot.process.buildActions.y = ConvertUnits(buildAction=LoadVector, inUnit="rad", outUnit="degree")
plot.process.buildActions.y.buildAction.vectorKey = "coord_dec"

In [ ]:
plot.finalize()
results = plot(diaSrc, band=data_id["band"], plotInfo=plot_info_dict)